# Lesson 2

In [9]:
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

import pandas as pd

data = sm.datasets.macrodata.load_pandas()
data = data.data
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [10]:
data = data[["realgdp", "realcons"]].pct_change().dropna()

gc_res = grangercausalitytests(data, 3)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.7248 , p=0.0000  , df_denom=198, df_num=1
ssr based chi2 test:   chi2=29.1600 , p=0.0000  , df=1
likelihood ratio test: chi2=27.2295 , p=0.0000  , df=1
parameter F test:         F=28.7248 , p=0.0000  , df_denom=198, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=18.9880 , p=0.0000  , df_denom=195, df_num=2
ssr based chi2 test:   chi2=38.9498 , p=0.0000  , df=2
likelihood ratio test: chi2=35.5873 , p=0.0000  , df=2
parameter F test:         F=18.9880 , p=0.0000  , df_denom=195, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=13.5015 , p=0.0000  , df_denom=192, df_num=3
ssr based chi2 test:   chi2=41.9812 , p=0.0000  , df=3
likelihood ratio test: chi2=38.0914 , p=0.0000  , df=3
parameter F test:         F=13.5015 , p=0.0000  , df_denom=192, df_num=3


In [16]:
df = pd.DataFrame.from_records(sm.datasets.macrodata.load().data)

def granger_causation_matrix(data, variables, test, verbose=False):
    x = pd.DataFrame(np.zeros((len(variables),len(variables))), columns=variables,index=variables)
    for c in x.columns:
        for r in x.index:
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose:
                print(f'Y = {r}, X= {c},P Values = {p_values}')
            min_p_value = np.min(p_values)
            x.loc[r,c] = min_p_value
            x.columns = [var + '_x' for var in variables]
            x.index = [var + '_y' for var in variables]

    return x

maxlag = 4

# granger_causation_matrix(df, df.columns, 'ssr_ftest', verbose=True)


C:\Users\adevr\AppData\Local\Temp\ipykernel_23396\483767198.py:1: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(sm.datasets.macrodata.load().data)


# Lesson 3

In [21]:
default_probabilities = np.ones(10) / 10
neg_corr = np.ones((10, 10)) * -0.1
neg_corr = neg_corr + np.eye(10) * 1.1
neg_corr

array([[ 1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [-0.1,  1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [-0.1, -0.1,  1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1,  1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1, -0.1,  1. , -0.1, -0.1, -0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1, -0.1, -0.1,  1. , -0.1, -0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1,  1. , -0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,  1. , -0.1, -0.1],
       [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,  1. , -0.1],
       [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,  1. ]])

In [22]:
np.all(np.linalg.eigvals(neg_corr) > 0)

np.True_

In [25]:
from frds.measures import DistressInsurancePremium

dip_model = DistressInsurancePremium(default_probabilities, neg_corr)
dip_value = dip_model.estimate(default_threshold=0.15)
print("Distress Insurance Premium:", dip_value)

Distress Insurance Premium: 0.07277788115999999


________________

In [26]:
pt6_corr = np.ones((10, 10)) * 0.6

zero_pt1 = np.eye(10) * 0.4

pt6_corr = pt6_corr + zero_pt1

pt6_corr

array([[1. , 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6],
       [0.6, 1. , 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6],
       [0.6, 0.6, 1. , 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, 1. , 0.6, 0.6, 0.6, 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, 0.6, 1. , 0.6, 0.6, 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, 0.6, 0.6, 1. , 0.6, 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1. , 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1. , 0.6, 0.6],
       [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1. , 0.6],
       [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1. ]])

In [27]:
dip_model = DistressInsurancePremium(default_probabilities, pt6_corr)
dip_value = dip_model.estimate(default_threshold=0.15)
print("Distress Insurance Premium:", dip_value)

Distress Insurance Premium: 0.3846018334
